In [57]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import OneHotEncoder
import pickle
import pandas as pd
import numpy as np

In [58]:
## load the trained model, scaler pickle, onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('One_hot_encoder.pkl', 'rb') as file:
    One_hot_encoder=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [59]:
import os
print(os.path.getsize("label_encoder_gender.pkl"))


258


In [60]:
#Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance':60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

input_df = pd.DataFrame([input_data])

In [61]:
# Don't fit again here in prediction.ipynb if you already fit+saved in experiments.ipynb
geo_encoder = One_hot_encoder.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(
    geo_encoder.toarray(),
    columns=One_hot_encoder.get_feature_names_out(['Geography'])
)
geo_encoded_df


c:\ANN Project Genai\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0

In [62]:
input_df

CreditScore Geography Gender  ...  HasCrCard  IsActiveMember  EstimatedSalary
0          600    France   Male  ...          1               1            50000

[1 rows x 10 columns]

In [63]:
## Encode categorical  variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

CreditScore Geography  Gender  ...  HasCrCard  IsActiveMember  EstimatedSalary
0          600    France       1  ...          1               1            50000

[1 rows x 10 columns]

In [64]:
## concatination with one hot enncoded  data 
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0          600       1  ...                0.0              0.0

[1 rows x 12 columns]

In [65]:
## Scaling the input data
scaled_input=scaler.transform(input_df)
scaled_input

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [66]:
## predict churn
prediction=model.predict(scaled_input)
prediction

1/1 [==============================] - 0s 347ms/step


array([[0.01814701]], dtype=float32)

In [67]:
prediction_prob=prediction[0][0]
prediction_prob

0.018147008

In [68]:
if prediction_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
